# Preprocessing of data into form suitable for input into RNNs. 

The basic form is: 

* A dataset is a list of tuples, with one tuple per stay. 
* Each stay tuple has a target Y, X_codes, X_pn, X_emar, X_fixed, which are themselves lists whose length is the number of prediction times in the stay. 
* Each element of X_fixed is the prediction time's demographics, stay, and vitals features. 
* Each element of X_codes is the prediction time's bag of codes, as indices into an embedding matrix starting at 1 (the 0 position is reserved for masking).  
* Each element of X_pn and X_emar respectively are the prediction time's bag of words, as indices into embedding matrices for progress notes and eMAR notes respectively, again starting at 1 with 0 reserved for masking. 

In [1]:
import os
import sys
import datetime
import pandas as pd
import pickle as pkl
import numpy as np
import scipy
import gc

from collections import namedtuple
from multiprocessing import Pool

from gensim.utils import simple_preprocess
from gensim.models import FastText

%load_ext autoreload
%autoreload 2

In [2]:
sys.path.append('/code')
from edge import data
from edge import patient_stays
from edge import diagnosis
from edge import meds
from edge import vitals
from edge import utils
from edge import stays
from edge import demographics

In [3]:
# Make a dir to put things
os.makedirs('/data/processed/rnn', exist_ok=True)

In [4]:
# Make dirs for pytorch data loader infrastructure
os.makedirs('/data/processed/rnn/data/train', exist_ok=True)
os.makedirs('/data/processed/rnn/data/val', exist_ok=True)
os.makedirs('/data/processed/rnn/data/test', exist_ok=True)

In [5]:
ptimes = pd.read_parquet('/data/raw/combined_ptimes.parquet')
ptimes.shape

(14694483, 4)

In [6]:
stays = pd.read_parquet('/data/raw/combined_stays.parquet')
stays.shape

(118127, 19)

In [7]:
used_stay_indices = ptimes.StayRowIndex.unique()
stays = stays.iloc[used_stay_indices]
print(len(stays))

116667


In [6]:
# We'll keep using this... 
grouped_ptimes = ptimes.groupby('StayRowIndex')

JOBS_DATA = []
for group_idx, group in grouped_ptimes: 
    # All we need are the indices into ptimes... 
    JOBS_DATA.append(group.index.values)

# Targets - 4 day rehosp

In [9]:
Y_flat = utils.get_rehosp_target(ptimes, stays, num_days=4)
np.mean(Y_flat)

0.015513509389884625

In [10]:
def _getTargetWorker(indices):
    return Y_flat[indices]

with Pool(os.cpu_count() - 4) as pool: 
    Y_list = pool.map(_getTargetWorker, JOBS_DATA)
len(Y_list)

116667

In [22]:
with open('/data/processed/rnn/target_4day_rehosp.pkl', 'wb') as f_out: 
    pkl.dump(Y_list, file=f_out)

# Fixed length features - stays, demographics, and vitals. 

In [11]:
stay_features = scipy.sparse.load_npz('/data/processed/combined_stay_features_csr.npz')
stay_features

<14694483x7 sparse matrix of type '<class 'numpy.float32'>'
	with 59069800 stored elements in Compressed Sparse Row format>

In [12]:
demo_features = scipy.sparse.load_npz('/data/processed/combined_demo_features_csr.npz')
demo_features

<14694483x4 sparse matrix of type '<class 'numpy.float32'>'
	with 29383026 stored elements in Compressed Sparse Row format>

In [13]:
vitals_features = scipy.sparse.load_npz('/data/processed/combined_vitals_features_csr.npz')
vitals_features

<14694483x72 sparse matrix of type '<class 'numpy.float32'>'
	with 164064183 stored elements in Compressed Sparse Row format>

In [14]:
fixed_length_features = scipy.sparse.hstack([stay_features, demo_features, vitals_features])
fixed_length_features = scipy.sparse.csr_matrix(fixed_length_features)

del stay_features
del demo_features
del vitals_features
gc.collect()

fixed_length_features

<14694483x83 sparse matrix of type '<class 'numpy.float32'>'
	with 252517009 stored elements in Compressed Sparse Row format>

In [15]:
def _getFixedLengthFeatures(indices): 
    return fixed_length_features[indices].todense()

with Pool(os.cpu_count() - 4) as pool: 
    X_fixed = pool.map(_getFixedLengthFeatures, JOBS_DATA)

len(X_fixed)

116667

In [21]:
with open('/data/processed/rnn/X_fixed.pkl', 'wb') as f_out: 
    pkl.dump(X_fixed, file=f_out)

In [16]:
del fixed_length_features
gc.collect()

22

# Code Embeddings

### Extract embedding matrix and mapping of codes to indices.  
Remember to leave index 0 for masking. 

In [172]:
code_model = FastText.load('/data/model/ft_combined_codes_d200.model')
code_model

In [173]:
code_vectors = code_model.wv
vocab = sorted(list(code_vectors.vocab.keys()))
len(vocab)

4785

In [174]:
embedding_dim = 200

num_codes = len(vocab)
embedding_matrix = np.zeros((num_codes+1, embedding_dim))
code_to_index = {k:(i+1) for i, k in enumerate(vocab)}
for code in vocab: 
    embedding_idx = code_to_index[code]
    wv = code_vectors[code]
    embedding_matrix[embedding_idx] = wv

np.save('/data/processed/rnn/code_embeddings.npy', embedding_matrix)
with open('/data/processed/rnn/code_to_index.pkl', 'wb') as f_out: 
    pkl.dump(code_to_index, file=f_out)
print('Done...')    

Done...


### Now build X_code

In [175]:
dx_features = scipy.sparse.load_npz('/data/processed/combined_dx_features_csr.npz')
rx_features = scipy.sparse.load_npz('/data/processed/combined_rx_features_csr.npz')
with open('/data/processed/combined_dx_colnames.pkl', 'rb') as f_in: 
    dx_names = pkl.load(f_in)
with open('/data/processed/combined_rx_colnames.pkl', 'rb') as f_in: 
    rx_names = pkl.load(f_in)
rx_names = np.array([s.replace(' ', '_') for s in rx_names])

code_features = scipy.sparse.csr_matrix(scipy.sparse.hstack([dx_features, rx_features]))
code_names = np.concatenate([dx_names, rx_names])
    
print('Done...')    

Done...


In [176]:
def _codeFeatureWorker(indices): 
    features = code_features[indices]
    rows, cols = np.nonzero(features > 0)
    retval = []
    for i in indices: 
        retval.append([])
    for i, j in zip(rows, cols): 
        code = code_names[j]
        if code in code_to_index: 
            code_idx = code_to_index[code]
            retval[i].append(code_idx)
    return retval

In [177]:
print('Starting jobs...')
with Pool(os.cpu_count() - 4) as pool: 
    X_codes = pool.map(_codeFeatureWorker, JOBS_DATA)
print('Done...')    

Starting jobs...
Done...


In [178]:
with open('/data/processed/rnn/X_codes.pkl', 'wb') as f_out: 
    pkl.dump(X_codes, file=f_out)

In [179]:
# Clean up a bit... 
del code_features
del dx_features
del rx_features
del code_model
del code_vectors
del embedding_matrix

gc.collect()

44

# eMAR notes
Similar recipe as for codes, but larger vocab of course. Have to deal with transforming note text too. 

In [8]:
# Load model, get vocab, map to indices, and save all
EMBEDDING_DIM = 200

emar_model = FastText.load('/data/model/ft_combined_emar_notes_d200.model')
word_vectors = emar_model.wv
vocab = list(word_vectors.vocab.keys())
print(f"Got vocab of {len(vocab)} words...")

print("Building embedding matrix...")
num_words = len(vocab)
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM))
word_to_index = {k:(i+1) for i, k in enumerate(vocab)}
for word in vocab: 
    embedding_idx = word_to_index[word]
    wv = word_vectors[word]
    embedding_matrix[embedding_idx] = word_vectors[word]

print("Saving embedding matrix and word to index map...")
np.save('/data/processed/rnn/emar_word_embeddings.npy', embedding_matrix)
with open('/data/processed/rnn/emar_word_to_index.pkl', 'wb') as f_out: 
    pkl.dump(word_to_index, file=f_out)    
    
print('Done...')

Got vocab of 26846 words...
Building embedding matrix...
Saving embedding matrix and word to index map...
Done...


### Now build X_emar

In [9]:
emar_notes = pd.read_parquet('/data/raw/combined_emar_notes.parquet')
emar_notes['Date'] = emar_notes.CreatedDate.dt.date
notes_subset = emar_notes[['MasterPatientID', 'FacilityID', 'ProgressNoteID', 'Date', 'NoteText']]
grp_cols = ['MasterPatientID', 'Date']
notes_merged_by_day = notes_subset.groupby(grp_cols).agg({'NoteText': lambda x: ' '.join(x)})
print('Done...')

Done...


In [10]:
ptimes_subset = ptimes[['MasterPatientID', 'FacilityID', 'StayRowIndex']].copy()
ptimes_subset['Date'] = (ptimes.PredictionTimestamp - pd.to_timedelta('1 day')).dt.date

merged_notes = ptimes_subset.merge(notes_merged_by_day, 
                                   how='left', 
                                   left_on=['MasterPatientID', 'Date'], 
                                   right_on=['MasterPatientID', 'Date'])

merged_notes['NoteText'] = merged_notes.NoteText.fillna('')

merged_notes.shape, ptimes_subset.shape

((14694483, 5), (14694483, 4))

In [12]:
def _notesWorker(indices): 
    note_texts = merged_notes.iloc[indices].NoteText.values
    retval = []
    for text in note_texts: 
        tokens = simple_preprocess(text)
        word_indices = [word_to_index[w] for w in tokens if w in word_to_index]
        # NB - we only use unique word indices...
        retval.append(list(set(word_indices)))
    return retval
    

In [13]:
with Pool(os.cpu_count() - 4) as pool: 
    X_emar = pool.map(_notesWorker, JOBS_DATA)
len(X_emar), len(X_emar[1])

(116667, 20)

In [14]:
with open('/data/processed/rnn/X_emar.pkl', 'wb') as f_out: 
    pkl.dump(X_emar, file=f_out)
print('Done...')

Done...


# Progress Notes

In [7]:
# Load model, get vocab, map to indices, and save all
EMBEDDING_DIM = 200

pn_model = FastText.load('/data/model/ft_combined_progress_notes_d200.model')
word_vectors = pn_model.wv
vocab = list(word_vectors.vocab.keys())
print(f"Got vocab of {len(vocab)} words...")

print("Building embedding matrix...")
num_words = len(vocab)
embedding_matrix = np.zeros((num_words+1, EMBEDDING_DIM))
word_to_index = {k:(i+1) for i, k in enumerate(vocab)}
for word in vocab: 
    embedding_idx = word_to_index[word]
    wv = word_vectors[word]
    embedding_matrix[embedding_idx] = word_vectors[word]

print("Saving embedding matrix and word to index map...")
np.save('/data/processed/rnn/pn_word_embeddings.npy', embedding_matrix)
with open('/data/processed/rnn/pn_word_to_index.pkl', 'wb') as f_out: 
    pkl.dump(word_to_index, file=f_out)    
    
print('Done...')

Got vocab of 97810 words...
Building embedding matrix...
Saving embedding matrix and word to index map...
Done...


### Now build X_pn

In [8]:
# Prepare progress notes by merging notes on same day for same patient... 
progress_notes = pd.read_parquet('/data/raw/combined_progress_notes.parquet')
progress_notes['Date'] = progress_notes.CreatedDate.dt.date
notes_subset = progress_notes[['MasterPatientID', 'FacilityID', 'ProgressNoteID', 'Date', 'NoteText']]
grp_cols = ['MasterPatientID', 'Date']
notes_merged_by_day = notes_subset.groupby(grp_cols).agg({'NoteText': lambda x: ' '.join(x)})
print('Done...')

Done...


In [9]:
ptimes_subset = ptimes[['MasterPatientID', 'FacilityID', 'StayRowIndex']].copy()
ptimes_subset['Date'] = (ptimes.PredictionTimestamp - pd.to_timedelta('1 day')).dt.date

merged_notes = ptimes_subset.merge(notes_merged_by_day, 
                                   how='left', 
                                   left_on=['MasterPatientID', 'Date'], 
                                   right_on=['MasterPatientID', 'Date'])
merged_notes['NoteText'] = merged_notes.NoteText.fillna('')
merged_notes.shape, ptimes_subset.shape

((14694483, 5), (14694483, 4))

In [10]:
def _notesWorker(indices): 
    note_texts = merged_notes.iloc[indices].NoteText.values
    retval = []
    for text in note_texts: 
        tokens = simple_preprocess(text)
        word_indices = [word_to_index[w] for w in tokens if w in word_to_index ]
        # NB - we only use unique word indices...
        retval.append(list(set(word_indices)))
    return retval

In [11]:
with Pool(os.cpu_count() - 4) as pool: 
    X_pn = pool.map(_notesWorker, JOBS_DATA)
len(X_pn), len(X_pn[1])

(116667, 20)

In [12]:
with open('/data/processed/rnn/X_pn.pkl', 'wb') as f_out: 
    pkl.dump(X_pn, file=f_out)
print('Done...')

Done...


In [ ]:
del merged_notes
del ptimes_subset
del progress_notes
gc.collect()

# Assemble tuples

In [6]:
# Load raw data... 
with open('/data/processed/rnn/X_pn.pkl', 'rb') as f_in: 
    X_pn = pkl.load(f_in)
with open('/data/processed/rnn/X_emar.pkl', 'rb') as f_in: 
    X_emar = pkl.load(f_in)
with open('/data/processed/rnn/X_codes.pkl', 'rb') as f_in: 
    X_codes = pkl.load(f_in)
with open('/data/processed/rnn/X_fixed.pkl', 'rb') as f_in: 
    X_fixed = pkl.load(f_in)
len(X_pn), len(X_emar), len(X_codes), len(X_fixed)

(116667, 116667, 116667, 116667)

In [23]:
with open('/data/processed/rnn/target_4day_rehosp.pkl', 'rb') as f_in: 
    Y = pkl.load(f_in)
len(Y)

116667

In [24]:
ALL_DATA = []
for Y_for_stay, X_for_stay in zip(Y, zip(X_fixed, X_codes, X_emar, X_pn)): 
    ALL_DATA.append((Y_for_stay, X_for_stay))
len(ALL_DATA)

116667

# Splits
Also, make tiny datasets for testing code. 

In [73]:
# Deal with left and right censoring. 
# Get rid of stays that start within 90 days of '2017-01-01'
MIN_DATE = pd.Timestamp('2017-01-01') + pd.to_timedelta('90 days')
print(f"Using min date {MIN_DATE}")

left_mask = (stays.StartDate >= MIN_DATE).values
print(f"Keeping {np.sum(left_mask)} stays out of {len(stays)}")

indices_to_keep = np.nonzero(left_mask)[0]
stays_to_use = stays.iloc[indices_to_keep]
print(f"Keeping {len(stays_to_use)} stays")

DATA = [ALL_DATA[idx] for idx in indices_to_keep]
print(len(DATA))

Using min date 2017-04-01 00:00:00
Keeping 94126 stays out of 116667
Keeping 94126 stays
94126


### Deal with right censoring
We should get rid of prediction times that are within 4 days of '2020-02-28', and that do not end in a rehosp event, but this is a bit complicated so defer until later.  This approximation should be fine.     

### Do the split

In [83]:
# train_start_date = '2017-01-01'
# train_end_date = '2019-07-31'
# val_end_date = '2019-11-30'
# test_end_date = '2020-02-28'

train_end = pd.Timestamp('2019-08-31')
val_end = pd.Timestamp('2019-11-30')

train_indices = np.nonzero((stays_to_use.StartDate <= train_end).values)[0]
val_indices = np.nonzero((stays_to_use.StartDate > train_end).values & (stays_to_use.StartDate <= val_end).values)[0]
test_indices = np.nonzero((stays_to_use.StartDate > val_end).values)[0]

print(len(train_indices), len(val_indices), len(test_indices))

78964 7371 7791


In [87]:
TRAIN_DATA = [DATA[idx] for idx in train_indices]
VAL_DATA = [DATA[idx] for idx in val_indices]
TEST_DATA = [DATA[idx] for idx in test_indices]
print(len(TRAIN_DATA), len(VAL_DATA), len(TEST_DATA))

with open('/data/processed/rnn/train_data.pkl', 'wb') as f_out: 
    pkl.dump(TRAIN_DATA, file=f_out)
with open('/data/processed/rnn/val_data.pkl', 'wb') as f_out: 
    pkl.dump(VAL_DATA, file=f_out)
with open('/data/processed/rnn/test_data.pkl', 'wb') as f_out: 
    pkl.dump(TEST_DATA, file=f_out)
    

78964 7371 7791


In [89]:
# Save stays for splits too... 
train_stays = stays_to_use.iloc[train_indices]
val_stays = stays_to_use.iloc[val_indices]
test_stays = stays_to_use.iloc[test_indices]
print(train_stays.StartDate.max(), val_stays.StartDate.min(), val_stays.StartDate.max(), test_stays.StartDate.min())

with open('/data/processed/rnn/train_stays.pkl', 'wb') as f_out: 
    pkl.dump(train_stays, file=f_out)
with open('/data/processed/rnn/val_stays.pkl', 'wb') as f_out: 
    pkl.dump(val_stays, file=f_out)
with open('/data/processed/rnn/test_stays.pkl', 'wb') as f_out: 
    pkl.dump(test_stays, file=f_out)
    
print(len(train_stays), len(val_stays), len(test_stays))

2019-08-31 2019-09-01 2019-11-30 2019-12-01
78964 7371 7791


In [90]:
# Also, get a tiny dataset for testing... 
N_tiny = 100
indices = np.random.choice(len(TRAIN_DATA), N_tiny, replace=False)
tiny_train = [TRAIN_DATA[i] for i in indices]
with open('/data/processed/rnn/tiny_train.pkl', 'wb') as f_out: 
    pkl.dump(tiny_train, file=f_out)
print(len(tiny_train))

100


# PyTorch DataLoader support
Since it provides out of box support for threading, etc, write things out like this was an image problem. 

In [35]:
import shutil

def prepare_pytorch_files(split='train'): 
    print("Processing data...")
    filepath = f'/data/processed/rnn/{split}_data.pkl'
    with open(filepath, 'rb') as f_in: 
        data = pkl.load(f_in)
    for i, d_i in enumerate(data): 
        if (i+1) % 1000 == 0: 
            print(f"Working on {i+1} / {len(data)}")
        Y, X = d_i
        X_fixed, X_codes, X_emar, X_pn = X
        X_fixed = np.array(X_fixed)
        X_codes = np.array([codes if len(codes) > 0 else [0] for codes in X_codes])
        X_emar = np.array([words if len(words) > 0 else [0] for words in X_emar])
        X_pn = np.array([words if len(words) > 0 else [0] for words in X_pn])
        dest_path = f'/data/processed/rnn/data/{split}/{i}.npz'
        np.savez(dest_path, 
                 Y=Y, 
                 X_fixed=X_fixed, 
                 X_codes=X_codes, 
                 X_emar=X_emar, 
                 X_pn=X_pn)
    
    print("Copying stays file...")
    stays_path = f'/data/processed/rnn/{split}_stays.pkl'
    new_stays_path = f'/data/processed/rnn/data/{split}/stays.pkl'
    shutil.copyfile(stays_path, new_stays_path)

    return

In [37]:
prepare_pytorch_files(split='train')
prepare_pytorch_files(split='val')
prepare_pytorch_files(split='test')

Processing data...
Working on 1000 / 78964
Working on 2000 / 78964
Working on 3000 / 78964
Working on 4000 / 78964
Working on 5000 / 78964
Working on 6000 / 78964
Working on 7000 / 78964
Working on 8000 / 78964
Working on 9000 / 78964
Working on 10000 / 78964
Working on 11000 / 78964
Working on 12000 / 78964
Working on 13000 / 78964
Working on 14000 / 78964
Working on 15000 / 78964
Working on 16000 / 78964
Working on 17000 / 78964
Working on 18000 / 78964
Working on 19000 / 78964
Working on 20000 / 78964
Working on 21000 / 78964
Working on 22000 / 78964
Working on 23000 / 78964
Working on 24000 / 78964
Working on 25000 / 78964
Working on 26000 / 78964
Working on 27000 / 78964
Working on 28000 / 78964
Working on 29000 / 78964
Working on 30000 / 78964
Working on 31000 / 78964
Working on 32000 / 78964
Working on 33000 / 78964
Working on 34000 / 78964
Working on 35000 / 78964
Working on 36000 / 78964
Working on 37000 / 78964
Working on 38000 / 78964
Working on 39000 / 78964
Working on 4000

### PyTorch Dataset experiments.

In [12]:
from edge.dnn import rnn_dataset

val_dataset = rnn_dataset.SaivaDataset('/data/processed/rnn/data/val', max_length=50)

In [13]:
Y, (X_fixed, X_codes, X_emar, X_pn) = val_dataset[0]

In [14]:
type(X_fixed)

torch.Tensor

In [15]:
X_fixed.size()

torch.Size([14, 83])

In [16]:
X_codes_inputs, X_codes_offsets = X_codes
X_codes_inputs.size(), X_codes_offsets.size()

(torch.Size([280]), torch.Size([14]))

In [17]:
Y.size()

torch.Size([14])

In [18]:
Y.device

device(type='cpu')

### Collating function for DataLoader

In [14]:
Y0, X0 = val_dataset[0]
Y1, X1 = val_dataset[1]
Y2, X2 = val_dataset[2]
batch = [(Y0, X0), (Y1, X1), (Y2, X2)]

In [15]:
from edge import dnn 

Y, X_fixed, X_codes, X_emar, X_pn, sizes = dnn.rnn_dataset.data_loader_collate_fn(batch)

In [16]:
sizes, Y.size(), X_fixed.size()

([14, 13, 50], torch.Size([50, 3]), torch.Size([50, 3, 83]))

In [17]:
X_fixed[:,0,0]

tensor([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
import torch

batch_size = 4
val_data = torch.utils.data.DataLoader(val_dataset, 
                                       batch_size=batch_size, 
                                       num_workers=4, 
                                       collate_fn=dnn.rnn_dataset.data_loader_collate_fn)
for batch_i, (Y, X_fixed, X_codes, X_emar, X_pn, sizes) in enumerate(val_data): 
    if batch_i == 5: 
        break

In [21]:
sizes

[50, 50, 41, 50]

In [22]:
fixed_size = X_fixed.size()[-1]
fixed_size

83

### Scratch pad for testing model architecture, etc. 

In [47]:
from edge.dnn.basic_model import BasicModel

model = BasicModel(fixed_size=fixed_size, code_size=10000, emar_size=50000, pn_size=100000, embed_dim=200, rnn_dim=128)
model

BasicModel(
  (code_embedding_bag): EmbeddingBag(10000, 200, mode=mean)
  (emar_embedding_bag): EmbeddingBag(50000, 200, mode=mean)
  (pn_embedding_bag): EmbeddingBag(100000, 200, mode=mean)
  (rnn): GRU(683, 128, num_layers=2)
  (dense_1): Linear(in_features=128, out_features=1, bias=True)
  (out_prob): Sigmoid()
)

In [48]:
dense_out, probs = model(X_fixed, X_codes, X_emar, X_pn, sizes, batch_size)

Got max length = 50 from sizes...
code embeddings have size torch.Size([200, 200])
new code embeddings have size torch.Size([50, 4, 200])
emar embeddings have size torch.Size([200, 200])
new emar embeddings have size torch.Size([50, 4, 200])
pn embeddings have size torch.Size([200, 200])
new pn embeddings have size torch.Size([50, 4, 200])
Combined inputs shape: torch.Size([50, 4, 683])
rnn output shape: torch.Size([50, 4, 128]), hidden shape: torch.Size([2, 4, 128])
logits has shape torch.Size([50, 4])


### >>> Output sizes! <<<

In [50]:
dense_out.size(), probs.size(), Y.size()

(torch.Size([50, 4]), torch.Size([50, 4]), torch.Size([50, 4]))

### Test a training step... 

### Try training for 3 steps... 

In [3]:
def _send_to_device(device, Y, X_fixed, X_codes, X_emar, X_pn): 
    Y.to(device)
    X_fixed.to(device)
    X_fixed.to(device)
    X_codes[0].to(device)
    X_codes[1].to(device)
    X_emar[0].to(device)
    X_emar[1].to(device)
    X_pn[0].to(device)
    X_pn[1].to(device)   

In [8]:
import torch
from edge.dnn import rnn_dataset
from edge.dnn.basic_model import BasicModel
from edge.dnn import utils

# Set seed! 
#torch.manual_seed(1)

# These should be from cmd line... 
max_length = 100  # Defined by dataset, etc... 
data_root_dir = '/data/processed/rnn/data'
batch_size = 32
num_data_workers = 4
lr = 0.001
num_epochs = 50
#num_epochs = 3

# Set up datasets and loaders.  
train_dataset = rnn_dataset.SaivaDataset(os.path.join(data_root_dir, 'train'), 
                                         max_length=max_length, 
                                         max_num=2000)
val_dataset = rnn_dataset.SaivaDataset(os.path.join(data_root_dir, 'val'), 
                                       max_length=max_length, 
                                       max_num=500)
test_dataset = rnn_dataset.SaivaDataset(os.path.join(data_root_dir, 'test'),
                                        max_length=max_length)

train_data = torch.utils.data.DataLoader(train_dataset, 
                                         batch_size=batch_size, 
                                         num_workers=num_data_workers, 
                                         collate_fn=rnn_dataset.data_loader_collate_fn)
val_data = torch.utils.data.DataLoader(val_dataset, 
                                       batch_size=batch_size, 
                                       num_workers=num_data_workers, 
                                       collate_fn=rnn_dataset.data_loader_collate_fn)
test_data = torch.utils.data.DataLoader(test_dataset, 
                                        batch_size=batch_size, 
                                        num_workers=num_data_workers, 
                                        collate_fn=rnn_dataset.data_loader_collate_fn)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device {device}")

fixed_size = 83
embeddings_dir = '/data/processed/rnn'

codes_file = os.path.join(embeddings_dir, 'code_embeddings.npy')
emar_file = os.path.join(embeddings_dir, 'emar_word_embeddings.npy')
pn_file = os.path.join(embeddings_dir, 'pn_word_embeddings.npy')
model = BasicModel(fixed_size=fixed_size, 
                   codes_file=codes_file, 
                   emar_file=emar_file, 
                   pn_file=pn_file, 
                   rnn_dim=200)
model.to(device)

# Set up objective function and optimizer. 
pos_weight = torch.Tensor([5.])
criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight) # Other options available...
optimizer = torch.optim.Adam(model.parameters(), lr=lr) 

Using device cpu


In [9]:
def train_epoch(model, optimizer, criterion, data, 
                device, print_every=10, max_iter=100): 
    
    total_num_iter = 0
    cumulative_loss = 0.    

    model.train()
    with torch.set_grad_enabled(True): 
        for batch_i, (Y, X_fixed, X_codes, X_emar, X_pn, sizes) in enumerate(data): 
            _send_to_device(device, Y, X_fixed, X_codes, X_emar, X_pn)
            batch_size = Y.size()[1]
            num_days = sum(sizes) 
            optimizer.zero_grad()

            mask = utils.get_mask_from_sizes(sizes)
            mask.to(device)  # Mask needs to be on device too I think?  

            logits, probs = model(X_fixed, X_codes, X_emar, X_pn, sizes, batch_size)
            loss = criterion(logits, Y)  # Loss is a Tensor of size (max_length, batch_size)...  
            masked_loss = loss * mask  # This is why mask should be on device I think?  
            scalar_loss = torch.sum(masked_loss) / torch.sum(mask)
            scalar_loss.backward()     # Back propagate gradients. 
            optimizer.step()    # Take the step...      

            cumulative_loss += scalar_loss.item() 
            total_num_iter += 1

            if (batch_i+1) % print_every == 0: 
                loss_right_now = cumulative_loss / total_num_days
                print(f"On iteration {batch_i+1} => {loss_right_now:.4f}")

    
    return cumulative_loss / total_num_iter
    

In [10]:
# Returns log loss, auroc, ap...  

from sklearn.metrics import roc_auc_score, average_precision_score

def evaluate_model(model, criterion, data, device): 

    Y_all = None
    Yhat_all = None
    total_num_iter = 0
    cumulative_loss = 0.  

    model.eval()
    with torch.set_grad_enabled(False): 
        for batch_i, (Y, X_fixed, X_codes, X_emar, X_pn, sizes) in enumerate(data): 
            _send_to_device(device, Y, X_fixed, X_codes, X_emar, X_pn)
            print(Y.device)
            batch_size = Y.size()[1] # dims are time x batch
            num_days = sum(sizes)
            
            mask = utils.get_mask_from_sizes(sizes)
            mask.to(device)  # Mask needs to be on device too I think?              
            
            logits, probs = model(X_fixed, X_codes, X_emar, X_pn, sizes, batch_size)
            loss = criterion(logits, Y) 
            masked_loss = loss * mask  
            scalar_loss = torch.sum(masked_loss) / torch.sum(mask)            

            # Accumulate loss
            cumulative_loss += scalar_loss.item() 
            total_num_iter += 1
            
            # Accumulate Y and Yhat given sizes (i.e., disregarding mask...)
            Y_batch = Y.detach().cpu().numpy()
            Yhat_batch = probs.detach().cpu().numpy()

            if Y_all is None: 
                Y_list, Yhat_list = [], []
            else: 
                Y_list, Yhat_list = [Y_all], [Yhat_all]
                
            for idx in range(Y_batch.shape[1]): 
                Y_list.append(Y_batch[:sizes[idx], idx])
                Yhat_list.append(Yhat_batch[:sizes[idx], idx])
            Y_all = np.concatenate(Y_list)
            Yhat_all = np.concatenate(Yhat_list)

    data_loss = cumulative_loss / total_num_iter
    auroc = roc_auc_score(Y_all, Yhat_all)
    ap = average_precision_score(Y_all, Yhat_all)
    return data_loss, auroc, ap, Y_all, Yhat_all
        

In [ ]:
best_i = 0
best_weights = None
which_metric = 'roc'
best_metric = np.Inf if which_metric == 'loss' else -np.Inf
patience = 5  # Run this many epochs with no improvement in val loss
epochs_since_improvement = 0
which_metric = 'roc'

def compare_stats(stats, which_metric, best_metric): 
    if which_metric == 'loss': 
        return stats[which_metric] < best_metric
    else: 
        return stats[which_metric] > best_metric
    
for epoch_i in range(num_epochs): 
    
    train_loss = train_epoch(model, optimizer, criterion, train_data, device, print_every=1000)
    print(f"Epoch {epoch_i+1} training loss: {train_loss:.4f}")
    
    # Evaluate and collect evaluation metrics.
#    val_loss, val_auroc, val_ap, Y, Yhat = evaluate_model(model, criterion, val_data, device)
    val_loss, val_auroc, val_ap, Y, Yhat = evaluate_model(model, criterion, train_data, device)
    
    stats = {'loss': val_loss, 
             'roc': val_auroc, 
             'ap': val_ap}
    print(f"Epoch {epoch_i+1} val loss: {val_loss:.4f} AUROC: {val_auroc:.4f} aP: {val_ap:.4f}")

    if compare_stats(stats, which_metric, best_metric): 
        best_metric = stats[which_metric]
        best_i = epoch_i + 1
        best_weights = copy.deepcopy(model.state_dict())
        epochs_since_improvement = 0
    else: 
        epochs_since_improvement += 1
    if epochs_since_improvement == patience: 
        break 
    

In [319]:
np.mean(Y)

0.04292714534483457

In [352]:
val_loss

0.5914622550139174

In [262]:
roc_auc_score(Y, Yhat)

0.5775934713931352

In [265]:
os.path.isdir('/data')

True

In [349]:
os.path.join('/', 'code')

'/code'

In [350]:
os.path.join('/', '/code')

'/code'